In [1]:
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as linalg
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from formulations.nda import NDA
from formulations.saaf import SAAF
from formulations.diffusion import Diffusion
from fe import *
from materials import Materials
from problem import Problem
from plot import plot, plot_1d
from solvers import Solver

ImportError: cannot import name 'plot_1d'

## Setup Problem

In [ ]:
mesh = 'symmetric-8'
mat = 'scattering1g'

In [ ]:
nodefile = "../test/test_inputs/" + mesh + ".node"
elefile = "../test/test_inputs/" + mesh + ".ele"
matfile = "../test/test_inputs/" + mat + ".mat"
grid = FEGrid(nodefile, elefile)
mats = Materials(matfile)
num_elts = grid.get_num_elts()
num_groups = mats.get_num_groups()
source = np.ones((num_groups, num_elts))

### Diffusion

In [ ]:
op = Diffusion(grid, mats)
solver = Solver(op)

In [ ]:
diffusion_phis = solver.solve(source)

### NDA

In [ ]:
op = NDA(grid, mats)
solver = Solver(op)

In [ ]:
nda_phis = solver.solve(source)

### SAAF

In [ ]:
op = SAAF(grid, mats)
solver = Solver(op)

In [ ]:
saaf_phis = solver.solve(source)

### Fix y to plot 1d solution

In [ ]:
y = 0.5

In [ ]:
diff_points = plot_1d(grid, diffusion_phis[0], y, 'x', display=True)
nda_points = plot_1d(grid, nda_phis[0], y, 'x', display=True)
saaf_points = plot_1d(grid, saaf_phis[0][0], y, 'x', display=True)

In [ ]:
plt.plot(diff_points[:, 0], diff_points[:, 1], label='diffusion')
plt.plot(nda_points[:, 0], nda_points[:, 1], label='nda')
plt.plot(saaf_points[:, 0], saaf_points[:, 1], label='saaf')
plt.legend()
plt.title("Scattering1g Solution at y=0.5")
plt.show()


# Compare Coarser Mesh to Fine Mesh

In [ ]:
mesh = 'symmetric-10'
mat = 'noscatter'

In [ ]:
nodefile = "../test/test_inputs/" + mesh + ".node"
elefile = "../test/test_inputs/" + mesh + ".ele"
matfile = "../test/test_inputs/" + mat + ".mat"
grid = FEGrid(nodefile, elefile)
mats = Materials(matfile)
num_elts = grid.get_num_elts()
num_groups = mats.get_num_groups()
source = np.ones((num_groups, num_elts))

### NDA

In [ ]:
op = NDA(grid, mats)
solver = Solver(op)

In [ ]:
nda_phis = solver.solve(source)

### SAAF

In [ ]:
op = SAAF(grid, mats)
solver = Solver(op)

In [ ]:
saaf_phis = solver.solve(source)

### Calculate Absorption Rate

In [ ]:
def calculate_absorption(grid, mats, phi, group_id):
    triang = grid.setup_triangulation()
    integral = 0
    for elt in range(grid.num_elts):
        midx = grid.get_mat_id(elt)
        sig_a = mats.get_siga(midx, group_id)
        g_nodes = grid.gauss_nodes(elt)
        phi_vals = grid.phi_at_gauss_nodes(triang, phi, g_nodes)
        integral += grid.gauss_quad(elt, sig_a*phi_vals[0])
    return integral

In [ ]:
nda_abs = calculate_absorption(grid, mats, nda_phis, 0)
nda_abs

In [ ]:
saaf_abs = calculate_absorption(grid, mats, saaf_phis[0], 0)
saaf_abs

## Compare SAAF & NDA as Mesh refines

In [ ]:
x = np.arange(4, 10)
err = []
for mesh in x:
    # Choose Input
    mesh = 'symmetric-%s' % (str(mesh))
    mat = 'scattering1g'

    # Setup Problem
    nodefile = "../test/test_inputs/" + mesh + ".node"
    elefile = "../test/test_inputs/" + mesh + ".ele"
    matfile = "../test/test_inputs/" + mat + ".mat"
    grid = FEGrid(nodefile, elefile)
    mats = Materials(matfile)
    num_elts = grid.get_num_elts()
    num_groups = mats.get_num_groups()
    source = np.ones((num_groups, num_elts))

    # Solve NDA
    op = NDA(grid, mats)
    solver = Solver(op)
    nda_phis = solver.solve(source)

    # Solve SAAF
    op = SAAF(grid, mats)
    solver = Solver(op)
    saaf_phis = solver.solve(source)

    # Compute Difference
    err.append(np.max(np.abs(nda_phis[0] - saaf_phis[0][0])))
err = np.array(err)
err

In [ ]:
x = np.arange(4, 11)
err = np.array()

In [ ]:
plt.plot(x, err, linestyle="--", marker='o')
plt.show()

In [ ]:
array([0.10806554, 0.11322092, 0.12015502, 0.12275716, 0.1220332 ,
       0.1240399 , 0.12415516])